# Hello World Workflow

# Step 1: Data Preparation & Put Data into ProjRepo

In [1]:
from sklearn import datasets
X, y= datasets.load_iris(return_X_y=True)

In [2]:
import numpy as np

In [3]:
filename_data_X = "/bd-fs-mnt/kc-repo/data/helloworld_X.csv"
filename_data_y = "/bd-fs-mnt/kc-repo/data/helloworld_y.csv"

In [4]:
np.savetxt(filename_data_X,X,delimiter=",")
np.savetxt(filename_data_y,y,delimiter=",")

# Step 2: Submit code to Training Cluster for Training

In [5]:
%attachments

  Cluster ID  Training Cluster    ML Engine
------------  ------------------  -----------
           3  pythoncpu           python


In [6]:
%%train 3

from sklearn import svm
import numpy as np

filename_data_X = "/bd-fs-mnt/kc-repo/data/helloworld_X.csv"
filename_data_y = "/bd-fs-mnt/kc-repo/data/helloworld_y.csv"


X = np.loadtxt(open(filename_data_X,"rb"),delimiter=",")
y = np.loadtxt(open(filename_data_y,"rb"),delimiter=",")

model = svm.SVC()
model.fit(X, y)


import pickle
# save the model to disk
filename = "/bd-fs-mnt/kc-repo/models/svm.pickle"
pickle.dump(model, open(filename, 'wb'))

History URL: http://hpecp-6.hpecplocal:10001/history/13


In [7]:
%logs --url http://hpecp-6.hpecplocal:10001/history/13

Job Status: Finished





# Step 3: Test the Trained model locally

In [11]:
X[90:91]

array([[5.5, 2.6, 4.4, 1.2]])

In [12]:
filename = "/bd-fs-mnt/kc-repo/models/svm.pickle"
model2 = pickle.load(open(filename, 'rb'))
model2.predict(X[90:91])

array([1.])

In [13]:
y[90]

1

# Step 4: Preparing the Scoring.py

In [10]:
import numpy as np
import os
import pickle
import json
import sys 

filename = "/bd-fs-mnt/kc-repo/models/svm.pickle"
model = pickle.load(open(filename, 'rb'))

# -----Reading from commandline from deployment as json
cli_input = {'x1': 5.5, 'x2': 2.6, 'x3': 4.4, 'x4': 1.2}
# cli_input = json.loads(sys.argv[1])

# my input for SVM 
my_input = np.array(list(cli_input.values()), ndmin=2)

myPredict = model.predict(my_input)
print("The prediction is ", myPredict[0])

The prediction is  1.0


## Write the file back to the Repository

In [16]:
%%writefile /bd-fs-mnt/kc-repo/code/SVMScoring2.py

import numpy as np
import os
import pickle
import json
import sys 

filename = "/bd-fs-mnt/kc-repo/models/svm.pickle"
model = pickle.load(open(filename, 'rb'))

# -----Reading from commandline from deployment as json
# cli_input = {'x1': 5.5, 'x2': 2.6, 'x3': 4.4, 'x4': 1.2}
cli_input = json.loads(sys.argv[1])

# my input for SVM 
my_input = np.array(list(cli_input.values()), ndmin=2)

myPredict = model.predict(my_input)
print("The prediction is ", myPredict[0])


Writing /bd-fs-mnt/kc-repo/code/SVMScoring2.py


# After Deployment

# Step 5: Use the Deployed Model

In [21]:
import requests
import pprint
import json

payload = {}
payload["use_scoring"] = True
payload["scoring_args"] = {'x1': 5.5, 'x2': 2.6, 'x3': 4.4, 'x4': 1.2}


url = "http://el4k-gateway.hpeilab.com:10023/svm-helloworld/v0.1/predict"
headers = {
  'Content-Type': 'application/json',
  'X-AUTH-TOKEN': 'og3YZ8cCqi'
}
response = requests.request("POST", url, headers=headers, data=json.dumps(payload))

json.loads(response.text)


{'input': "{''x1'': 5.5, ''x2'': 2.6, ''x3'': 4.4, ''x4'': 1.2}",
 'log_url': 'http://el4k-gateway.hpeilab.com:10023/logs/N15_9',
 'node': 'hpecp-14.hpecplocal',
 'output': 'The prediction is  1.0\n\n',
 'pid': '2254',
 'qid': 'N15_9',
 'request_url': 'http://el4k-gateway.hpeilab.com:10023/history/N15_9',
 'status': 'Finished'}